In [1]:
import numpy as np
import pandas as pd

In [2]:
start_path  = '원본데이터/'
mid_path  = '전처리필요데이터/'
end_path = '분석용데이터/'
extension = '.csv'
encoding  = 'cp949'

In [ ]:
use_p = ['당현', '월계', '화랑', '노원역', '상계1', '마들']

def police_location_csv():
    p_loc = pd.read_csv(start_path+'경찰서정보'+extension , encoding=encoding)
    p_loc = p_loc.drop(['연번', '경찰서', '구분', '전화번호'], axis=1)
    p_loc = p_loc.loc[p_loc['관서명'].isin(use_p), :]
    p_loc.to_csv(mid_path+'경찰서정보.csv', index=False, encoding=encoding)

#police_location_csv()

In [ ]:
X_DIFF_M = 0.00001125
Y_DIFF_M = 0.00000898
POLICE_RAD_M = 900
GRID_DIFF_M  = 90

def make_grid(p_x, p_y):
    # 정사각 격자점 생성
    x = np.linspace(p_x - X_DIFF_M*POLICE_RAD_M, p_x + X_DIFF_M*POLICE_RAD_M, 2*POLICE_RAD_M//GRID_DIFF_M + 1)
    y = np.linspace(p_y - Y_DIFF_M*POLICE_RAD_M, p_y + Y_DIFF_M*POLICE_RAD_M, 2*POLICE_RAD_M//GRID_DIFF_M + 1)
    X, Y = np.meshgrid(x, y)
    grid = np.stack([X, Y], axis = 2).reshape(-1, 2)
    # 원형 필터링
    dist = (grid - [p_x, p_y]) / [X_DIFF_M, Y_DIFF_M] # grid에는 경,위도가 담겼으므로 원점으로 옮기고 미터로 환산
    dist = np.linalg.norm(dist, axis = 1)
    grid = grid[dist <= POLICE_RAD_M]
    #데이터프레임화
    grid = pd.DataFrame(grid, columns=['경도', '위도'])
    return grid

def make_grid_csv():
    p_loc = pd.read_csv(end_path+'경찰서정보.csv')
    for i in p_loc.index:
        grid = make_grid(p_loc.loc[i, '경도'], p_loc.loc[i, '위도'])
        grid.to_csv(mid_path+p_loc.loc[i, '이름']+extension, index=False)

#make_grid_csv()

In [21]:
import folium

df = pd.read_csv(mid_path+'당현'+extension)

# 지도 중심을 서울(첫 번째 위치)로 설정
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=15)

# 데이터프레임의 각 위치에 마커 추가
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['위도'], row['경도']],
        radius = 5,
        color = '#FF0000'
    ).add_to(m)

# 지도 출력
m
